In [1]:
import time
import sys
import random
from datetime import timedelta
from uuid import getnode
from enum import Enum

In [37]:
# define lower and upper limits of random delay for response to attendance messages
RAND_LOWER = 0
RAND_UPPER = 0.5
WAIT_FOR_ATTENDANCE_SEC = (
    2  # how many seconds to wait for message before taking leader position
)
ATTENDANCE_RESPONSE_SEC = 1.5  # send duration for follower responses
SEND_LIST_DELAY = 0.1  # how long to wait between list messages
WAIT_FOR_CHECK_IN_RESPONSE = 0.1  # how long to wait for response from device
CHECK_IN_DELAY = 0.1  # how long to delay before sending next check in message

# define bit masks and shifts based on message details
class ActionCodes(Enum):
    ATTENDANCE = 0b1000
    RESPONSE = 0b0001
    SONG = 0b0010
    FIRST_LIST = 0b0011  # first list message (not necessary if we have followers add leader to list by default when attendance msg is heard)
    N_LIST = 0b0100  # following list messages
    CHECK_IN = 0b0101
    DELETE = 0b0110


class MessageBits(Enum):
    # messages are formatted with action as least significant bits
    # option bits are most significant
    # 9223372036854775807 is max 48 bit integer
    # 65535 is max 16 bit integer
    ACTION_LEN = 4
    ACTION_SHIFT = 0
    ACTION_MASK = 0xF << ACTION_SHIFT
    FOLLOW_ADDR_LEN = 48
    FOLLOW_ADDR_SHIFT = ACTION_SHIFT + ACTION_LEN
    FOLLOW_ADDR_MASK = 0xFFFFFFFFFFFF << FOLLOW_ADDR_SHIFT
    LEADER_ADDR_LEN = 48
    LEADER_ADDR_SHIFT = FOLLOW_ADDR_SHIFT + FOLLOW_ADDR_LEN
    LEADER_ADDR_MASK = 0xFFFFFFFFFFFF << LEADER_ADDR_SHIFT
    OPTION_LEN = 16
    OPTION_SHIFT = LEADER_ADDR_SHIFT + LEADER_ADDR_LEN
    OPTION_MASK = 0xFFFF << OPTION_SHIFT


class Message:
    def __init__(self, msg: int):
        self.action = self.bit_masking(
            msg, MessageBits.ACTION_MASK, MessageBits.ACTION_SHIFT
        )
        self.leader_addr = self.bit_masking(
            msg, MessageBits.LEADER_ADDR_MASK, MessageBits.LEADER_ADDR_SHIFT
        )
        self.follow_addr = self.bit_masking(
            msg, MessageBits.FOLLOW_ADDR_MASK, MessageBits.FOLLOW_ADDR_SHIFT
        )
        self.options = self.bit_masking(
            msg, MessageBits.OPTION_MASK, MessageBits.OPTION_SHIFT
        )

        # negatives are trasmitted as two's complement
        if self.options == (1 << MessageBits.OPTION_LEN.value) - 1:
            self.options = -1

    def bit_masking(self, msg, mask, shift):
        return (msg & mask.value) >> shift.value

    def __str__(self) -> str:
        return f"Message object w/ Action: {bin(self.action)},\nLeader Address: {hex(self.leader_addr)}\nFollower Address: {hex(self.follow_addr)}\nOptions: {self.options}"


def create_message(
    action: ActionCodes, follower_addr: int, leader_addr: int, options=None
):
    msg = 0
    msg |= action.value << MessageBits.ACTION_SHIFT.value
    msg |= follower_addr << MessageBits.FOLLOW_ADDR_SHIFT.value
    msg |= leader_addr << MessageBits.LEADER_ADDR_SHIFT.value
    if not options == None:
        msg |= options << MessageBits.OPTION_SHIFT.value
    return msg

def remove_length_byte(msg):
    mask = (1 << (msg.bit_length() - 8)) - 1
    msg &= mask
    return msg

In [5]:
1 << MessageBits.OPTION_SHIFT.value

1267650600228229401496703205376

In [14]:
print(hex(MessageBits.LEADER_ADDR_MASK.value))

0xffffffffffff0000000000000


In [31]:
msg = create_message(ActionCodes.ATTENDANCE, 0, getnode(), -1)
print(bin(msg))
print(msg.bit_length())

-0b1000000100010000110010111100100111110111010001001111111111111111111111111111111111111111111111111000
100


In [38]:
print(Message(msg))

Message object w/ Action: 0b1000,
Leader Address: 0x7eef343608bb
Follower Address: 0x0
Options: -1


In [27]:
# 65535 >> 15
1 << 16

65536

In [46]:
a = (255<<16).to_bytes(length=4, byteorder='big')
h = a.hex()
print(h)

print(h[2:])

00ff0000
ff0000
